<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

Из чего состоит датасет:

- Признаки: пол, возраст, зарплата застрахованного, количество членов семьи.

- Целевой признак: количество страховых выплат клиенту за последние 5 лет.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
data = pd.read_csv('/datasets/insurance.csv')
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


На первый взгляд тип у столбцов "Пол" (это должен быть булевый тип) и "Возраст" (в большинстве случаев люди указывают возраст целиком, а не дробно) неверные, нужно их изменить. В остальном же всё нормально, даже пропуски отсутствуют.

In [4]:
data['Возраст'] = data['Возраст'].astype('int64')
data['Пол'] = data['Пол'].astype('bool')

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Посмотрим как будут выглядеть предсказания в исходной задаче:

$$
a = Xw = X(X^T X)^{-1}X^Ty
$$

**Ответ:** Качество линейной регрессии не изменится

**Обоснование:** 


- Умножим матрицу признаков X на матрицу Р и получим:
$$
w = (XP)P^{-1}w = (XP)w'
$$

$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$

$$
w' = (P^T (X^T X) P)^{-1} (XP)^T y
$$

$$
w' = P^{-1}(X^T X)^{-1}(P^T)^{-1}P^TX^Ty
$$

$$
w' = P^{-1}(X^T X)^{-1}X^Ty
$$

- Теперь подставим w в а:
$$
a = XPw = XPP^{-1}(X^T X)^{-1}X^Ty = X(X^T X)^{-1}X^Ty
$$

## Алгоритм преобразования

**Алгоритм**
1. Возьмем случайно сгенерированную матрицу $P$ размера $n*n$, определим $n$ по числу признаков в $X-1$;
2. Проверим существование обратной матрицы $P$;
3. Если обратная матрица существует, то умножим исходную матрицу $X$ на матрицу $P$.
4. Для проверки определим метрику R2, и сравним результат на оригинальном и преобразованном датасетах.

**Обоснование**

Согласно доказательству выше, при умножении матрицы признаков $X$ на произвольную, обратимую матрицу $P$, качество линейной регрессии останется неизменным.

In [5]:
def transformation(data, target, seed=88):
    
    np.random.seed(seed)
    n = data.shape[1] - 1
    
    try:
        P = np.random.normal(size=(n, n))
        np.linalg.inv(P)
    except:
        P = np.random.normal(size=(n, n))
        np.linalg.inv(P)
    
    select = data.drop(target, axis=1).columns
    
    data[select] = data[select] @ P
    
    return data

## Проверка алгоритма

In [6]:
def ml_run(data, target, secret=False):
    
    if secret:
        X = transformation(data, target).drop(target, axis=1)
    else:
        X = data.drop(target, axis=1)
        
    y = data[target]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
 
    model = LinearRegression()
    model.fit(X_train, y_train)
    prediction = model.predict(X_test)
    
    return r2_score(y_test, prediction)

In [7]:
target = 'Страховые выплаты'

print(f'R2 метрика для оригинальной матрицы: {ml_run(data, target, secret=False):.5f}')
print(f'R2 метрика для преобразованой матрицы: {ml_run(data, target, secret=True):.5f}')

R2 метрика для оригинальной матрицы: 0.42548
R2 метрика для преобразованой матрицы: 0.42548


Качество линейной регресии не изменилось, значит мы смогли доказать, что при умножении матрицы признаков $X$ на произвольную, обратимую матрицу $P$, качество линейной регрессии останется неизменным.